In [40]:
import numpy as np
import math
import random
import time

In [41]:
def openFile():
    query = {}
    doc = {}
    root_path = './data'

    with open(f'{root_path}/query_list.txt', 'r') as file:
        query_list = file.read()
        for file_name in query_list.split('\n'):
            try:
                file_path = f'{root_path}/queries/{file_name}.txt'
                with open(file_path, 'r') as f:
                    query[file_name] = f.read().lower()
            except Exception as e:
                print(e)

    with open(f'{root_path}/doc_list.txt', 'r') as file:
        doc_list = file.read()
        for file_name in doc_list.split('\n'):
            try:
                file_path = f'{root_path}/docs/{file_name}.txt'
                with open(file_path, 'r') as f:
                    doc[file_name] = f.read().lower()
            except Exception as e:
                print(e)

    print(len(query))
    print(len(doc))
    
    return query, doc

In [42]:
query_dict, doc_dict = openFile()

[Errno 2] No such file or directory: './data/queries/.txt'
[Errno 2] No such file or directory: './data/docs/.txt'
100
14955


In [43]:
def cal_word_count(doc_dict):
    word_dict = {}   # word: count
    all_word_len = 0 # 計算 total word length in document

    for doc_name, doc in doc_dict.items():
        all_word_len += len(doc.split(' '))
        
        for word in doc.split(' '):
            if word_dict.get(word, 0):
                word_dict[word] += 1
            else:
                word_dict[word] = 1
        
    return word_dict, all_word_len

In [44]:
### Calculate word count & total word length

word_dict, all_word_len = cal_word_count(doc_dict)

print(len(word_dict))
print(all_word_len)

111449
7059938


In [45]:
### new word dict: 減少 word 的數量
query_word = []
for _, value in query_dict.items():
    query_word.append(value.split(' '))
query_word = sum(query_word, [])

# select word if word in query or tf > 40
new_word_dict = {}
for word in list(word_dict.keys()):
    if word in query_word or word_dict[word] > 100:
        new_word_dict[word] = word_dict[word]

print(len(new_word_dict))

6127


In [46]:
def cal_tf(doc_dict, new_word_dict):
    tf_dict = {}   # tf[doc_index][word]
    word2id =  {}  # 對應, used in EM algorithm
    id2word = {}   # 對應, used in query
    
    doc_index, word_index = 0, 0
    
    for _, doc in doc_dict.items():
        tf_dict[doc_index] = {}
        
        for word in doc.split(' '):
            if new_word_dict.get(word, 0): # 如果在 new word dict, 才計算 tf
                if tf_dict[doc_index].get(word, 0):
                    tf_dict[doc_index][word] += 1
                else:
                    tf_dict[doc_index][word] = 1
                    word2id[word] = word_index   # 初始化後建對應表
                    id2word[word_index] = word
                    word_index += 1
        doc_index += 1
        
    return tf_dict, word2id, id2word

In [47]:
### Calculate tf & build mapping dict
tf_dict, word2id, id2word = cal_tf(doc_dict, new_word_dict)

In [48]:
for i, j in tf_dict.items():
    print(i)
    print(j)
    break

0
{'languag': 1, 'f': 4, 'p': 2, '105': 1, 'english': 1, 'articl': 1, 'type': 1, 'bfn': 1, '106': 1, 'report': 1, 'text': 1, 'march': 2, '17': 2, 'african': 1, 'nation': 2, 'congress': 1, 'anc': 4, 'presid': 2, 'nelson': 1, 'mandela': 5, 'statement': 3, 'elect': 1, 'ralli': 2, 'wednesday': 1, 'night': 1, 'parti': 2, 'mp': 1, 'dr': 4, 'said': 4, 'thursday': 1, 'mr': 4, 'told': 1, 'thousand': 1, 'support': 1, 'senior': 1, 'state': 1, 'offici': 1, 'perpetr': 1, 'violenc': 3, 'leader': 1, 'also': 2, 'charg': 1, 'hope': 1, 'april': 1, '27': 1, 'suffici': 1, 'evid': 1, 'show': 1, 'third': 2, 'forc': 2, 'intellig': 1, 'public': 1, 'commiss': 1, 'conduct': 1, 'thorough': 1, 'investig': 1, 'found': 1, 'proof': 1, 'exist': 1, 'common': 1, 'knowledg': 1, 'involv': 1, 'far': 1, 'major': 1, 'case': 1, 'intimid': 2, 'throughout': 1, 'countri': 1, 'seriou': 1, 'would': 1, 'acknowledg': 1, 'main': 1, 'someth': 1, 'hard': 1, 'hit': 1, 'undermin': 1, 'nobel': 1, 'peac': 1, 'conclud': 1}


In [49]:
### Calculate BG word
BG_word = {}
for word, count in new_word_dict.items():
    BG_word[word] = count / all_word_len

In [50]:
def initialParameter(doc_len, word_len, K):
    T_w = np.random.random([K, word_len])
    d_T = np.random.random([doc_len, K])
    
    for k in range(0, K):
        normalization = sum(T_w[k, :])
        for i in range(0, word_len):
            T_w[k, i] /= normalization

    for j in range(0, doc_len):
        normalization = sum(d_T[j, :])
        for k in range(0, K):
            d_T[j, k] /= normalization
            
    return T_w, d_T

In [51]:
doc_len, word_len = len(doc_dict), len(new_word_dict)

# number of TOPIC
K = 2

# T_W[topic][word] : p(wi|Tk)
# D_T[doc][topic] : p(Tk|dj)
# e_step[doc][word][topic] : p(Tk|wi,dj)
T_w, d_T = initialParameter(doc_len, word_len, K)
print(T_w.shape)
print(d_T.shape)

e_step = np.zeros([doc_len,word_len,K])
print(e_step.shape)

(2, 6127)
(14955, 2)
(14955, 6127, 2)


In [52]:
def EStep(doc_len, word_len, tf_dict, K, T_w, d_T, e_step):
    print('EStep: ', time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
    
    for i in range(0, doc_len):
        for j in range(0, word_len):
            
            word = id2word[j]
            if not tf_dict[i].get(word, 0):
                continue
                
            denominator = 0
            
            for k in range(0, K):
                e_step[i][j][k] = T_w[k][j] * d_T[i][k]
                denominator += e_step[i][j][k]
                
            if denominator == 0:
                for k in range(0, K):
                    e_step[i][j][k] = 0
            else:
                for k in range(0, K):
                    e_step[i][j][k] /= denominator
    
    return e_step

In [53]:
def MStep(doc_len, word_len, tf_dict, K, T_w, d_T, e_step):
    print('MStep: ', time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
    
    ### update T_w : p(wi|Tk)
    for k in range(0, K):
        denominator = 0
        
        for j in range(0, word_len):
            T_w[k][j] = 0
            
            for i in range(0, doc_len):
                word = id2word[j]
                if tf_dict[i].get(word, 0):
                    T_w[k][j] += tf_dict[i][word] * e_step[i][j][k]
            denominator += T_w[k][j]
            
        if denominator == 0:
            for j in word_len:
                T_w[k][j] = 1.0 / word_len
        else:
            for j in range(0, word_len):
                T_w[k][j] /= denominator
                
    ### update d_T : p(Tk|dj)
    for i in range(0, doc_len):
        for k in range(0, K):
            d_T[i][k] = 0
            denominator = 0
            
            for j in range(0, word_len):
                word = id2word[j]
                if tf_dict[i].get(word, 0):
                    d_T[i][k] += tf_dict[i][word] * e_step[i][j][k]
                    denominator += tf_dict[i][word]
                
            if denominator == 0:
                d_T[i][k] = 1.0 / K
            else:
                d_T[i][k] /= denominator
                
    return T_w, d_T

In [54]:
def Likelihood(doc_len, word_len, tf_dict, K, T_w, d_T):
    likelihood = 0
    
    for i in range(0, doc_len):
        for j in range(0, word_len):
            tmp = 0
            
            for k in range(0, K):
                tmp += T_w[k][j] * d_T[i][k]
                
            if tmp > 0:
                word = id2word[j]
                if tf_dict[i].get(word, 0):
                    likelihood += tf_dict[i][word] * math.log(tmp, 10)
                
    return likelihood

In [55]:
def EM_algorithm(doc_len, word_len, tf_dict, K, T_w, d_T, e_step):
    Iteration = 20
    threshold = 100.0
    oldLoglikelihood = 1
    newLoglikelihood = 1

    for i in range(0, Iteration):
        e_step = EStep(doc_len, word_len, tf_dict, K, T_w, d_T, e_step)
        T_w, d_T = MStep(doc_len, word_len, tf_dict, K, T_w, d_T, e_step)
        newLoglikelihood = Likelihood(doc_len, word_len, tf_dict, K, T_w, d_T)
        
        print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
        
        if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
            break
        oldLoglikelihood = newLoglikelihood
    
    return T_w, d_T

In [56]:
def PLSA_model(query, doc_dict, tf_dict, BG_word, K, T_w, d_T):
    alpha, beta = 0.8, 0.2
    score_dict = {}
    
    for i in range(0, doc_len):
        doc_name = doc_list[i]
        doc = doc_dict[doc_name]
        score = 0
        
        for word in query.split(' '):
            tf = tf_dict[i].get(word, 0)  # 將 word 轉成 score
            
            tmp = 0
            id_word = word2id[word]
            for k in range(0, K):
                tmp += T_w[k][id_word] * d_T[i][k]
            
            first = alpha * (tf / len(doc))
            second = beta * tmp
            third = (1 - alpha - beta) * BG_word[word]
            
            score = first + second + third
            
        score_dict[doc_name] = score
    
    rank = sorted(score_dict.items(), key=lambda x: x[1], reverse = True) # 根據分數做排序
    return rank

In [ ]:
T_w, d_T = EM_algorithm(doc_len, word_len, tf_dict, K, T_w, d_T, e_step)

EStep:  2020-11-24 12:39:16
MStep:  2020-11-24 12:40:29


In [ ]:
f = open('ans.txt', 'w')
string = 'Query,RetrievedDocuments\n'

for _id, _query in query_dict.items():
    rank = PLSA_model(_query, doc_dict, tf_dict, BG_word, K, T_w, d_T)

    string += _id + ','
    for i, doc in enumerate(rank):
        string += doc[0] + ' '
        
        if i == 999:
            break
    string += '\n'
    
f.write(string)
f.close()
print('done')